# Importação dos dados do Banco de Dados "clvhealth_jcafb_2025_15"

## Leitura das Credenciais

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/jcafb_2025.db')

hosts = pd.read_sql_query('SELECT * FROM host', conn, index_col='name')

conn.close()

creds = lambda index, col_name : hosts.at[index, col_name]


## Parâmetros globais

In [2]:
import logging
# import host
import ssl

logging.basicConfig(level=logging.DEBUG)
_logger = logging.getLogger(__name__)

ssl._create_default_https_context = ssl._create_unverified_context
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


In [3]:
from xmlrpc import client

host_name = 'tkl-odoo15-jcafb25-vm'
# host_name = 'tkl-odoo16-vm-18'

server_url = creds(host_name, 'server')
db_name = creds(host_name, 'dbname')
username = creds(host_name, 'user')
# password = creds(host_name, 'user_pw')
password = creds(host_name, 'user_apikey')


## Importação dos dados de "res.country"

In [4]:
res_country_fields = ['id', 'name', 'code']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_country_objects = models.execute_kw(db_name, user_id, password,
                                            'res.country', 'search_read',
                                            [search_domain, res_country_fields],
                                            {}
                                           )
    res_country = pd.DataFrame(res_country_objects)

    # print(res_country)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_country')
    conn.commit()

    # res_country.to_sql('res_country', conn, if_exists='replace', index=False)
    res_country.to_sql('res_country', conn, if_exists='append', index=False)

    conn.close()


## Importação dos dados de "res.partner"

In [5]:
res_partner_fields = ['id', 'name', 'type']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_partner_objects = models.execute_kw(db_name, user_id, password,
                                            'res.partner', 'search_read',
                                            [search_domain, res_partner_fields],
                                            {}
                                           )
    res_partner = pd.DataFrame(res_partner_objects)

    # print(res_partner)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_partner')
    conn.commit()

    # res_partner.to_sql('res_partner', conn, if_exists='replace', index=False)
    res_partner.to_sql('res_partner', conn, if_exists='append', index=False)

    conn.close()


## Importação dos dados de "res.company"

In [6]:
res_company_fields = ['id', 'name']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_company_objects = models.execute_kw(db_name, user_id, password,
                                            'res.company', 'search_read',
                                            [search_domain, res_company_fields],
                                            {}
                                           )
    res_company = pd.DataFrame(res_company_objects)

    # print(res_company)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_company')
    conn.commit()

    # res_company.to_sql('res_company', conn, if_exists='replace', index=False)
    res_company.to_sql('res_company', conn, if_exists='append', index=False)

    conn.close()


## Importação dos dados de "res.users"

In [7]:
# res_users_fields = ['id', 'name', 'partner_id', 'company_id', 'parent_id', 'tz', 'lang', 'country_id',
#                     'login', 'password', 'groups_id', 'active', 'image_1920']
res_users_fields = ['id', 'name', 'partner_id', 'company_id', 'parent_id', 'tz', 'lang', 'country_id',
                    'login', 'password', 'active', 'image_1920']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_users_objects = models.execute_kw(db_name, user_id, password,
                                          'res.users', 'search_read',
                                          [search_domain, res_users_fields],
                                          {}
                                         )
    res_users = pd.DataFrame(res_users_objects)

    for i, row in res_users.iterrows():

        if row['partner_id']:
            res_users['partner_id'].values[i] = row['partner_id'][0]
        else:
            res_users['partner_id'].values[i] = None

        if row['company_id']:
            res_users['company_id'].values[i] = row['company_id'][0]
        else:
            res_users['company_id'].values[i] = None

        if row['parent_id']:
            res_users['parent_id'].values[i] = row['parent_id'][0]
        else:
            res_users['parent_id'].values[i] = None

        if row['country_id']:
            res_users['country_id'].values[i] = row['country_id'][0]
        else:
            res_users['country_id'].values[i] = None

    # print(res_users)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_users')
    conn.commit()

    # res_users.to_sql('res_users', conn, if_exists='replace', index=False)
    res_users.to_sql('res_users', conn, if_exists='append', index=False)

    conn.close()


## Importação dos dados de "clv_phase"

In [8]:
clv_phase_fields = ['id', 'name', 'description']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    clv_phase_objects = models.execute_kw(db_name, user_id, password,
                                          'clv.phase', 'search_read',
                                          [search_domain, clv_phase_fields],
                                          {}
                                         )
    clv_phase = pd.DataFrame(clv_phase_objects)

    for i, row in clv_phase.iterrows():

        if row['description'] is False:
            clv_phase['description'].values[i] = None

    # print(clv_phase)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM clv_phase')
    conn.commit()

    # clv_phase.to_sql('clv_phase', conn, if_exists='replace', index=False)
    clv_phase.to_sql('clv_phase', conn, if_exists='append', index=False)

    conn.close()


## Importação dos dados de "clv_patient_marker"

In [9]:
clv_patient_marker_fields = ['id', 'name', 'description', 'color', 'active']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    clv_patient_marker_objects = models.execute_kw(db_name, user_id, password,
                                          'clv.patient.marker', 'search_read',
                                          [search_domain, clv_patient_marker_fields],
                                          {}
                                         )
    clv_patient_marker = pd.DataFrame(clv_patient_marker_objects)

    for i, row in clv_patient_marker.iterrows():

        if row['description'] is False:
            clv_patient_marker['description'].values[i] = None

    # print(clv_patient_marker)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM clv_patient_marker')
    conn.commit()

    # clv_patient_marker.to_sql('clv_patient_marker', conn, if_exists='replace', index=False)
    clv_patient_marker.to_sql('clv_patient_marker', conn, if_exists='append', index=False)

    conn.close()


## Importação dos dados de "clv_patient"

In [10]:
clv_patient_fields = ['id', 'name', 'code', 'gender', 'birthday', 'phase_id',
                      'address_name', 'street', 'street_name', 'street_number', 'street2', 'street_number2',
                      'zip', 'city_id', 'state_id', 'country_id',
                      'mobile', 'email', 'marker_ids'
                     ]

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    clv_patient_objects = models.execute_kw(db_name, user_id, password,
                                            'clv.patient', 'search_read',
                                            [search_domain, clv_patient_fields],
                                            {}
                                           )
    clv_patient = pd.DataFrame(clv_patient_objects)
    clv_patient.insert(clv_patient.columns.get_loc("phase_id") + 1, 'phase', None)
    clv_patient.insert(clv_patient.columns.get_loc("city_id") + 1, 'city', None)
    clv_patient.insert(clv_patient.columns.get_loc("state_id") + 1, 'country_state', None)
    clv_patient.insert(clv_patient.columns.get_loc("country_id") + 1, 'country', None)
    clv_patient.insert(clv_patient.columns.get_loc("marker_ids") + 1, 'markers', None)

    for i, row in clv_patient.iterrows():

        if row['phase_id']:
            clv_patient['phase_id'].values[i] = row['phase_id'][0]
            clv_patient['phase'].values[i] = row['phase_id'][1]
        else:
            clv_patient['phase_id'].values[i] = None

        if not row['street_name']:
            clv_patient['street_name'].values[i] = None

        if not row['street_number']:
            clv_patient['street_number'].values[i] = None

        if not row['street_number2']:
            clv_patient['street_number2'].values[i] = None

        if not row['street2']:
            clv_patient['street2'].values[i] = None

        if row['city_id']:
            clv_patient['city_id'].values[i] = row['city_id'][0]
            clv_patient['city'].values[i] = row['city_id'][1]
        else:
            clv_patient['city_id'].values[i] = None

        if row['state_id']:
            clv_patient['state_id'].values[i] = row['state_id'][0]
            clv_patient['country_state'].values[i] = row['state_id'][1]
        else:
            clv_patient['state_id'].values[i] = None

        if row['country_id']:
            clv_patient['country_id'].values[i] = row['country_id'][0]
            clv_patient['country'].values[i] = row['country_id'][1]
        else:
            clv_patient['country_id'].values[i] = None

        if not row['mobile']:
            clv_patient['mobile'].values[i] = None

        if not row['email']:
            clv_patient['email'].values[i] = None

        if row['marker_ids'] != []:
            clv_patient['marker_ids'].values[i] = str(row['marker_ids'])
            markers = False
            marker_ids = row['marker_ids']
            for val in marker_ids:
                index = clv_patient_marker.id[clv_patient_marker.id == val].index.tolist()[0]
                if markers is False:
                    markers = clv_patient_marker.at[index, 'name']
                else:
                    markers = ';' + clv_patient_marker.at[index, 'name']
            clv_patient['markers'].values[i] = markers
        else:
            clv_patient['marker_ids'].values[i] = None

    # print(clv_patient)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM clv_patient')
    conn.commit()

    # clv_patiBaseExceptionent.to_sql('clv_patient', conn, if_exists='replace', index=False)
    clv_patient.to_sql('clv_patient', conn, if_exists='append', index=False)

    conn.close()
